# Objective:
Here we compute all features except text based features. Text based features are computed separately in a different notebook. Features computed here includes
1. Mean encodings on item_id, shop_id, category_id etc.
2. Lag features.
3. Rolling features.
4. Shop's city based features (latitude, longitude, city importance)
5. Oldness features based on item_id, shop_id: For how long an item has been trading in the market, or for how long an item has been trading in a specific shop.
6. Price based features.

Here, control flow is that we compute all features inside of **ModelData class** except mean encodings. Mean encodings are computed at the end.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
print(os.listdir('../input/addingzerototrain/'))
# Any results you write to the current directory are saved as output.

['addingzerototrain', 'competitive-data-science-predict-future-sales']
['__notebook__.ipynb', '__output__.json', 'train_with_zero.hdf', 'custom.css', '__results__.html']


In [2]:
DATA_FNAME='DATA.hdf'

In [3]:
sales = pd.read_hdf('../input/addingzerototrain/train_with_zero.hdf','df')
# sales = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
categ = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')
ITEM_EN = []

In [4]:
sales.loc[sales['item_price'] < 0, 'item_price'] = 0
sales.loc[sales['item_cnt_day'] > 300, 'item_cnt_day'] = 300
sales.loc[sales['item_cnt_day'] < 0, 'item_cnt_day'] = 0

In [5]:
# English translation of categories.
CATEGORIES_EN = [
    'PC - Headsets / Headphones',
    'Accessories - PS2',
    'Accessories - PS3',
    'Accessories - PS4',
    'Accessories - PSP',
    'Accessories - PSVita',
    'Accessories - XBOX 360',
    'Accessories - XBOX ONE',
    'Tickets (Digital)',
    'Delivery of goods',
    'Game Consoles - PS2',
    'Game Consoles - PS3',
    'Game Consoles - PS4',
    'Gaming Consoles - PSP',
    'Game Consoles - PSVita',
    'Gaming Consoles - XBOX 360',
    'Gaming Consoles - XBOX ONE',
    'Game Consoles - Other',
    'Games - PS2',
    'Games - PS3',
    'Games - PS4',
    'Games - PSP',
    'Games - PSVita',
    'Games - XBOX 360',
    'Games - XBOX ONE',
    'Games - Accessories for games',
    'Android Games - Digital',
    'MAC Games - Digit',
    'PC Games - Additional Editions',
    'PC Games - Collectible Editions',
    'PC Games - Standard Editions',
    'PC Games - Digital',
    'Payment cards (Cinema, Music, Games)',
    'Payment Cards - Live!',
    'Payment Cards - Live! (Numeral)',
    'Payment Cards - PSN',
    'Payment Cards - Windows (Digital)',
    'Cinema - Blu-Ray',
    'The Movie - Blu-Ray 3D',
    'Cinema - Blu-Ray 4K',
    'Cinema - DVD',
    'Cinema - Collection',
    'Books - Artbooks, encyclopedias',
    'Books - Audiobooks',
    'Books - Audiobooks (Figure)',
    'Books - Audiobooks 1C',
    'Books - Business Literature',
    'Books - Comics, Manga',
    'Books - Computer Literature',
    'Books - Methodical materials 1C',
    'Books - Postcards',
    'Books - Cognitive Literature',
    'Книги - Путеводители',
    'Books - Fiction',
    'Books - The Figure',
    'Music - Local Production CD',
    'Music - CD branded production',
    'Music - MP3',
    'Music - Vinyl',
    'Music - Music Video',
    'Music - Gift Edition',
    'Gifts - Attributes',
    'Gifts - Gadgets, Robots, Sports',
    'Gifts - Soft Toys',
    'Gifts - Board Games',
    'Gifts - Board Games (Compact)',
    'Gifts - Cards, stickers',
    'Gifts - Development',
    'Gifts - Certificates, Services',
    'Gifts - Souvenirs',
    'Gifts - Souvenirs (in a hitch)',
    'Gifts - Bags, Albums, Mouse pads',
    'Gifts - Figures',
    'Programs - 1C: Enterprise 8',
    'Programs - MAC (Digit)',
    'Programs - Home and Office',
    'Programs - Home and Office (Digital)',
    'Programs - Educational',
    'Programs - Educational (Figure)',
    'Service',
    'Service Tickets',
    'Clean media (spire)',
    'Clean Media (Piece)',
    'Batteries',
]
# English translation of shops names.
SHOPS_EN = [
    '! Yakutsk Ordzhonikidze, 56 fran',
    ' ! Yakutsk Shopping Center "Central" fran',
    ' Adygea shopping center "Mega"',
    ' Balashikha October "Kinomir"',
    ' Volzhsky shopping center "Volga Mall"',
    ' Vologda shopping center "Marmalade"',
    ' Voronezh (Plekhanovskaya, 13)',
    ' Voronezh TRC "Maksimir"',
    ' Voronezh TRC City-Park "Grad"',
    ' Outbound Trading',
    ' Zhukovsky st. Chkalov 39m?',
    ' Zhukovsky st. Chkalov 39m²',
    ' Online Emergency Store',
    ' Kazan TC "Bahetle"',
    ' Kazan TC "ParkHouse" II',
    ' Kaluga TRC "XXI century"',
    ' Kolomna TC "Rio"',
    ' Krasnoyarsk shopping center "Vzletka Plaza"',
    ' Krasnoyarsk shopping center "June"',
    ' Kursk TC "Pushkinsky"',
    ' Moscow "Sale"',
    ' Moscow MTRC "Afi Mall"',
    ' Moscow Shop S21',
    ' Moscow TC "Budenovsky" (pav.A2)',
    ' Moscow TC "Budenovsky" (pav. K7)',
    ' Moscow TRK "Atrium"',
    ' Moscow TC "Areal" (Belyaevo)',
    ' Moscow TC "MEGA Belaya Dacha II"',
    ' Moscow TC "MEGA Teply Stan" II',
    ' Moscow shopping center "New Century" (Novokosino)',
    ' Moscow TPP "Perlovski"',
    ' Moscow shopping center "Semenovsky"',
    ' Moscow shopping center "Silver House"',
    ' Mytishchi TRC "XL-3"',
    ' N.Novgorod SEC "RIO"',
    ' N.Novgorod SEC "Fantastic"',
    ' Novosibirsk SEC "Gallery Novosibirsk"',
    ' Novosibirsk TC "Mega"',
    ' Omsk TC "Mega"',
    ' RostovNaDon TRK "Megacenter Horizon"',
    ' RostovNaDonu TRK "Megacenter Horizon" Ostrovnoy',
    ' RostovNaDonu TC "Mega"',
    ' St. Petersburg TC "Nevsky Center"',
    ' St. Petersburg TC "Sennaya"',
    ' Samara TP "Melody"',
    ' Samara TC "ParkHaus"',
    ' Sergiev Posad TC 7YA',
    ' Surgut SEC "City Mall"',
    ' Tomsk SEC "Emerald City"',
    ' Tyumen SEC "Crystal"',
    ' Tyumen shopping center "Goodwin"',
    ' Tyumen shopping center "Green Coast"',
    ' Ufa TC "Central"',
    ' Ufa shopping center "Family" 2',
    ' Khimki TC "Mega"',
    ' Digital warehouse 1C-Online',
    ' Chekhov SEC "Carnival"',
    ' Yakutsk Ordzhonikidze, 56',
    ' Yakutsk shopping center "Central"',
    ' Yaroslavl shopping center "Altair',
]


## City data for shops: area, city_id, importance, latitude, longitude

In [6]:
from geopy import distance
from geopy.geocoders import Nominatim

CITY_DATA = {
    'Калуга': {
        'type': 'city',
        'importance': 0.657295410002972,
        'area': 358.911060027607,
        'lat': 54.5101087,
        'lon': 36.2598115
    },
    'Волжский': {
        'type': 'city',
        'importance': 0.550220481385513,
        'area': 438.60566745306613,
        'lat': 48.782102,
        'lon': 44.7779843
    },
    'Воронеж': {
        'type': 'city',
        'importance': 0.7045722199183749,
        'area': 2087.0423692794884,
        'lat': 51.6605982,
        'lon': 39.2005858
    },
    'Жуковский': {
        'type': 'city',
        'importance': 0.5444576798578901,
        'area': 84.74916146073933,
        'lat': 55.5972801,
        'lon': 38.1199863
    },
    'Самара': {
        'type': 'city',
        'importance': 0.696449565028584,
        'area': 2205.1176282331953,
        'lat': 53.198627,
        'lon': 50.113987
    },
    'Коломна': {
        'type': 'city',
        'importance': 0.648802312001607,
        'area': 149.8134437525239,
        'lat': 55.0938743,
        'lon': 38.7670121
    },
    'Якутск': {
        'type': 'city',
        'importance': 0.654131338136001,
        'area': 304.64719883743965,
        'lat': 62.027287,
        'lon': 129.732086
    },
    'Ярославль': {
        'type': 'city',
        'importance': 0.7065300207968069,
        'area': 455.605256943589,
        'lat': 57.6263877,
        'lon': 39.8933705
    },
    'Балашиха': {
        'type': 'city',
        'importance': 0.604005896721309,
        'area': 341.3399072537641,
        'lat': 55.8036225,
        'lon': 37.9646488
    },
    '!Якутск': {
        'type': 'city',
        'importance': 0.654131338136001,
        'area': 304.64719883743965,
        'lat': 62.027287,
        'lon': 129.732086
    },
    'Новосибирск': {
        'type': 'city',
        'importance': 0.728185699386452,
        'area': 977.968425123674,
        'lat': 55.0282171,
        'lon': 82.9234509
    },
    'Тюмень': {
        'type': 'city',
        'importance': 0.672385913300304,
        'area': 1029.734170100062,
        'lat': 57.153534,
        'lon': 65.542274
    },
    'Москва': {
        'type': 'city',
        'importance': 0.922316290384526,
        'area': 2220.463036723328,
        'lat': 55.7504461,
        'lon': 37.6174943
    },
    'Томск': {
        'type': 'city',
        'importance': 0.680307551446071,
        'area': 378.01721837627406,
        'lat': 56.488712,
        'lon': 84.952324
    },
    'Казань': {
        'type': 'city',
        'importance': 0.741972620362528,
        'area': 1317.0415739017471,
        'lat': 55.7823547,
        'lon': 49.1242266
    },
    'Курск': {
        'type': 'city',
        'importance': 0.676179816540548,
        'area': 399.62069154089585,
        'lat': 51.739433,
        'lon': 36.179604
    },
    'Уфа': {
        'type': 'city',
        'importance': 0.695077392339603,
        'area': 1721.7276870129429,
        'lat': 54.726288,
        'lon': 55.947727
    },
    'Вологда': {
        'type': 'city',
        'importance': 0.661524874526942,
        'area': 214.3430466663635,
        'lat': 59.218876,
        'lon': 39.893276
    },
    'Ростов На Дону': {
        'type': 'city',
        'importance': 0.7893194127943861,
        'area': 811.0162296285571,
        'lat': 47.2213858,
        'lon': 39.7114196
    },
    'Адыгея': {
        'type': 'administrative',
        'importance': 0.692337665336592,
        'area': 27295.264468374502,
        'lat': 44.6939006,
        'lon': 40.1520421
    },
    'Химки': {
        'type': 'city',
        'importance': 0.614359398958108,
        'area': 230.77301628863356,
        'lat': 55.8892847,
        'lon': 37.4449896
    },
    'Чехов': {
        'type': 'town',
        'importance': 0.49824054756674396,
        'area': 38.796925301165096,
        'lat': 55.1426603,
        'lon': 37.4545328
    },
    'Мытищи': {
        'type': 'city',
        'importance': 0.617191804459829,
        'area': 95.08539485314338,
        'lat': 55.9094928,
        'lon': 37.7339358
    },
    'Красноярск': {
        'type': 'city',
        'importance': 0.698696821640559,
        'area': 836.7336288346606,
        'lat': 56.0090968,
        'lon': 92.8725147
    },
    'Сергиев': {
        'type': 'city',
        'importance': 0.529283352524288,
        'area': 88.864113189393,
        'lat': 56.3153529,
        'lon': 38.1358208
    },
    'Сургут': {
        'type': 'city',
        'importance': 0.617507029143067,
        'area': 315.54371943139705,
        'lat': 61.254032,
        'lon': 73.3964
    },
    'СПб': {
        'type': 'city',
        'importance': 0.837718425910333,
        'area': 1137.355369029224,
        'lat': 59.938732,
        'lon': 30.316229
    },
    'Омск': {
        'type': 'city',
        'importance': 0.706863149131586,
        'area': 2424.721167579471,
        'lat': 54.991375,
        'lon': 73.371529
    },
    'Н.Новгород': {
        'type': 'signal',
        'importance': 0.11100000000000002,
        'area': 6.734885040711671e-05,
        'lat': 57.1810745,
        'lon': 45.1382139
    },
    'Ни́жний Но́вгород': {
        'type': 'city',
        'importance': 0.972093333761689,
        'area': 570.3511705993303,
        'lat': 56.328571,
        'lon': 44.003506
    },
}


def area_city(geocode):
    lat = list(map(float, geocode.raw['boundingbox'][:2]))
    lon = list(map(float, geocode.raw['boundingbox'][2:]))
    length = distance.distance((lat[0], lon[0]), (lat[0], lon[1])).km
    breadth = distance.distance((lat[0], lon[0]), (lat[1], lon[0])).km
    return length * breadth

def add_city_name(shops_df):
    shops_df['city'] = shops_df.shop_name.apply(lambda x: x.split(' ')[0])
    shops_df.loc[shops_df.city =='РостовНаДону', 'city'] = 'Ростов На Дону'
    shops_df.loc[shops_df.city =='Н.Новгород', 'city'] = 'Ни́жний Но́вгород'


class CityData:
    def __init__(self, shops_df):
        self._shops_df = shops_df.copy()
        self.data = {}
        self._geolocator = None

        add_city_name(self._shops_df)
        self._invalid_cities = ['Интернет-магазин', 'Выездная', 'Цифровой']


    def run(self):
        if self._geolocator is None:
            self._geolocator = Nominatim(
                user_agent="sales prediction_{}".format(np.random.rand()))
        cities_ru = set(self._shops_df.city.values)
        for city in cities_ru:
            if city in self._invalid_cities or city in self.data:
                continue

            location = self._geolocator.geocode(city + ', Россия')
            assert location.raw['type'] in ['city', 'town', 'administrative']
            self.data[city] = {
                'type': location.raw['type'],
                'importance': location.raw['importance'],
                'area': area_city(location),
                'lat': location.latitude,
                'lon': location.longitude
            }

            print(city, location.raw['type'],
                  round(location.raw['importance'], 2),
                  (location.latitude, location.longitude))



In [7]:
import pandas as pd
import numpy as np
# from city_data import CITY_DATA, add_city_name
from sklearn.preprocessing import LabelEncoder


def add_city_data_features(X_df, shops_df):
    """
    Latitude, Longitude, Area and Importance of cities is added
    """
    shops_df = shops_df.copy()
    add_city_name(shops_df)
    shops_df['city_id'] = LabelEncoder().fit_transform(shops_df['city']).astype(np.int16)
    city_features = ['city_id']
    for key in ['lat', 'lon', 'importance', 'area']:
        feature = 'city_' + key
        shops_df[feature] = shops_df['city'].apply(lambda x: CITY_DATA[x][key] if x in CITY_DATA else -1).astype(
            np.float32)
        city_features.append(feature)

    X_df = X_df.reset_index()
    X_df = pd.merge(X_df, shops_df[city_features + ['shop_id']], how='left', on='shop_id')
    X_df.set_index('index', inplace=True)
    return X_df


# if __name__ == '__main__':
#     from constants import SALES_FPATH, SHOPS_FPATH
#     sales_df = pd.read_csv(SALES_FPATH)
#     shops_df = pd.read_csv(SHOPS_FPATH)

#     print(add_city_data_features(sales_df, shops_df).head())


## Id features processing

In [8]:
import pandas as pd


class IdFeatures:
    ABSENT_ITEM_ID_VALUE = -1000

    def __init__(self, sales_df: pd.DataFrame, items_df: pd.DataFrame):

        self._sales_df = sales_df
        self._items_df = items_df

        # first time occuring features
        self._item_fm_df = None
        self._item_shop_fm_df = None

    def _fit_first_time_occuring_features(self):
        assert 'orig_item_id' in self._sales_df

        temp_df = self._sales_df[self._sales_df.item_cnt_day > 0][['orig_item_id', 'shop_id', 'date_block_num']]

        self._item_fm_df = temp_df.groupby(['orig_item_id'])['date_block_num'].min().to_frame('fm').reset_index()
        self._item_shop_fm_df = temp_df.groupby(['orig_item_id',
                                                 'shop_id'])['date_block_num'].min().to_frame('fm').reset_index()
        assert 'orig_item_id' in self._item_fm_df
        assert 'orig_item_id' in self._item_shop_fm_df

    def get_fm_features(self, df, item_id_and_shop_id=False):
        """
        Adds  first month features to df.
        df must have ['orig_item_id','shop_id','date_block_num'] columns
        """
        assert 'orig_item_id' in df
        if self._item_fm_df is None:
            self._fit_first_time_occuring_features()

        merge_df = self._item_shop_fm_df if item_id_and_shop_id else self._item_fm_df
        on_columns = ['orig_item_id', 'shop_id'] if item_id_and_shop_id else ['orig_item_id']
        f_nm_prefix = '_'.join(on_columns) + '_'
        df = pd.merge(df.reset_index(), merge_df, on=on_columns, how='left').set_index('index')

        old_col = f_nm_prefix + 'oldness'
        fm_col = f_nm_prefix + 'is_fm'

        df[old_col] = df['date_block_num'] - df['fm']
        # We will set oldness to 0 for which we don't have the data.
        df[old_col] = df[old_col].fillna(0).astype(int)

        df[fm_col] = df[old_col] == 0

        return df.drop('fm', axis=1)


## Numeric features

In [9]:
from multiprocessing import Pool
import pandas as pd
from datetime import datetime, date
from typing import Union


def get_datetime(date_block_num):
    year = 2013 + date_block_num // 12
    month = date_block_num % 12 + 1
    return datetime(year, month, 1)


def get_date_block_num(dt: Union[date, datetime]):
    m = dt.month
    y = dt.year
    return (y - 2013) * 12 + (m - 1)


def run(fn_args):
    fn, args, kwargs = fn_args
    return fn(*args, **kwargs)


def compute_concurrently(args, process_count=4):

    with Pool(processes=process_count) as pool:
        output = pool.map(run, args)

    df = pd.concat(output, axis=1)
    return df


def get_items_in_market(sales_df, block_num):
    """
    Returns list of item_ids which are traded from the first day of given month
    """
    last_trading_month = sales_df.groupby(['item_id'])['date_block_num'].max()
    first_trading_month = sales_df.groupby(['item_id'])['date_block_num'].min()
    it1 = set(last_trading_month[last_trading_month >= block_num].index.tolist())
    it2 = set(first_trading_month[first_trading_month <= block_num].index.tolist())
    output = list(it1.intersection(it2))
    output.sort()
    return output


def get_shops_in_market(sales_df, block_num):
    """
    Returns list of shop_ids which are open in that month.
    """
    last_trading_month = sales_df.groupby(['shop_id'])['date_block_num'].max()
    first_trading_month = sales_df.groupby(['shop_id'])['date_block_num'].min()
    it1 = set(last_trading_month[last_trading_month >= block_num].index.tolist())
    it2 = set(first_trading_month[first_trading_month <= block_num].index.tolist())
    output = list(it1.intersection(it2))
    output.sort()
    return output


## Rolling features: aggregate on past num_months data.

In [10]:
import numpy as np
import pandas as pd
from sortedcontainers import SortedList


def nmonths_features(sales_df, col_name, num_months, quantiles):
    """
    It does not use that month's data to compute features. it uses previous months data.
    Args:
        sales_df: it  is sorted by shop_id, item_id and date_block_num.
            Also, shop_item_group value changes when either the shop or the item changes.
    """
    assert num_months >= 1
    columns = ['sum', 'min', 'max'] + ['{}_q'.format(q) for q in quantiles]
    columns = ['{}_{}M_{}'.format(col_name, num_months, c) for c in columns]

    # For fast processing
    values = sales_df[col_name].values
    indices = sales_df.index.tolist()

    output_data = np.zeros((len(indices), len(columns)))
    s_list = SortedList()

    rolling_sum = 0
    tail_index_position = 0
    head_index_position = -1
    tail_index = indices[tail_index_position]
    tail_dbn = sales_df.iloc[0]['date_block_num']

    cur_group = sales_df.iloc[0]['shop_item_group']
    for head_index in indices:
        head_group = sales_df.at[head_index, 'shop_item_group']
        head_index_position += 1
        head_dbn = sales_df.at[head_index, 'date_block_num']

        if head_group != cur_group:
            s_list = SortedList()
            cur_group = head_group
            tail_index = head_index
            tail_index_position = head_index_position
            tail_dbn = sales_df.at[head_index, 'date_block_num']
            rolling_sum = 0
        else:
            if head_index_position >= 1:
                item = values[head_index_position - 1]
                s_list.add(item)
                rolling_sum += item

        while head_dbn - tail_dbn > num_months:

            value_to_be_removed = values[tail_index_position]
            s_list.remove(value_to_be_removed)
            rolling_sum -= value_to_be_removed

            tail_index_position += 1
            tail_index = indices[tail_index_position]
            tail_dbn = sales_df.at[tail_index, 'date_block_num']

        if len(s_list) == 0:
            output_data[head_index_position, :] = [0] * output_data.shape[1]
        else:
            # compute values on data starting at tail_index_position and ending at head_index_position, both inclusive.
            quants = [s_list[max(0, int(len(s_list) * q) - 1)] for q in quantiles]
            output_data[head_index_position, :] = [rolling_sum, s_list[0], s_list[-1]] + quants

    return pd.DataFrame(output_data, columns=columns, index=sales_df.index)


### Lagged Features
Given any feature, it will return what was the feature's value n months back, where n is a parameter specified in `lags`

In [11]:
import pandas as pd


def add_lagged_features(df, feature_name, lags=[3, 6, 12]):
    df = df.reset_index()
    new_features = []
    for month_lag in lags:
        lag_f = df[['item_id', 'shop_id', feature_name, 'date_block_num']].copy()
        lag_f['date_block_num'] = lag_f['date_block_num'] + month_lag - 1
        new_fname = '{}_{}M'.format(feature_name, month_lag)
        new_features.append(new_fname)
        lag_f.rename({feature_name: new_fname}, inplace=True, axis=1)
        df = pd.merge(df, lag_f, how='left', on=['item_id', 'shop_id', 'date_block_num'])
        print('{} Month lagged feature computed.'.format(month_lag))

    df[new_features] = df[new_features].fillna(-10)
    return df.set_index('index')


### Price based features

In [12]:
import numpy as np
import pandas as pd

INVALID_VALUE = -10


def get_price_features(sales_df, X_df):
    return _get_price_features(sales_df, X_df, 'item_price')


def get_dollar_value_features(sales_df, X_df):
    sales_df['dollar_value'] = (sales_df['item_price'] * sales_df['item_cnt_day']).astype(np.float32)
    output_df = _get_price_features(sales_df, X_df, 'dollar_value')
    sales_df.drop('dollar_value', axis=1, inplace=True)
    return output_df


def _get_price_features(sales_df, X_df, price_col):
    """
    sales_df is monthly
    """
    # use original sales data.
    msg = 'X_df has >1 recent months data. To speed up the process, we are just handling 1 month into the future case'
    assert X_df.date_block_num.max() <= sales_df.date_block_num.max() + 1, msg

    sales_df = sales_df[sales_df.item_cnt_day > 0].copy()
    sales_df.loc[sales_df[price_col] < 0, price_col] = 0

    grp = sales_df.groupby(['item_id', 'date_block_num'])[price_col]
    # std for 1 entry should be 0. std for 0 entry should be -10
    std = grp.std().fillna(0).unstack()
    std[sales_df.date_block_num.max() + 1] = 0
    std = std.sort_index(axis=1).fillna(method='ffill', axis=1).shift(1, axis=1).fillna(INVALID_VALUE)
    std_col = 'std_{}'.format(price_col)
    std = std.stack().to_frame(std_col).reset_index()

    avg_price = grp.mean().unstack()
    avg_price[sales_df.date_block_num.max() + 1] = 0
    avg_price = avg_price.sort_index(axis=1).fillna(method='ffill', axis=1).shift(1, axis=1).fillna(INVALID_VALUE)
    avg_col = 'avg_{}'.format(price_col)
    avg_price = avg_price.stack().to_frame(avg_col).reset_index()

    last_price_df = sales_df[['item_id', 'shop_id', 'date_block_num', price_col]].copy()
    last_price_df['date_block_num'] += 1
    last_pr_col = 'last_{}'.format(price_col)
    last_price_df.rename({price_col: last_pr_col}, inplace=True, axis=1)

    # index
    X_df = X_df.reset_index()

    # item_id price
    X_df = pd.merge(X_df, avg_price, on=['item_id', 'date_block_num'], how='left')
    X_df[avg_col] = X_df[avg_col].fillna(INVALID_VALUE)

    # shop_id item_id coupled price
    X_df = pd.merge(X_df, last_price_df, on=['item_id', 'shop_id', 'date_block_num'], how='left')

    X_df[last_pr_col] = X_df[last_pr_col].fillna(X_df[avg_col])

    # stdev
    X_df = pd.merge(X_df, std, on=['item_id', 'date_block_num'], how='left')
    X_df[std_col] = X_df[std_col].fillna(INVALID_VALUE)

    X_df.set_index('index', inplace=True)
    X_df[[std_col, last_pr_col, avg_col]] = X_df[[std_col, last_pr_col, avg_col]].astype(np.float32)

    price_category = np.log1p(X_df[last_pr_col] - INVALID_VALUE)

    # CATEGORY FEATURES.
    categ_col = 'category_{}'.format(price_col)
    subcateg_col = 'sub_category_{}'.format(price_col)

    avg_categ_col = 'avg_category_{}'.format(price_col)
    avg_subcateg_col = 'avg_sub_category_{}'.format(price_col)

    X_df[categ_col] = price_category.astype(int)
    X_df[subcateg_col] = (price_category - X_df[categ_col]).astype(np.float32)

    avg_price_category = np.log1p(X_df[avg_col] - INVALID_VALUE)
    X_df[avg_categ_col] = avg_price_category.astype(int)
    X_df[avg_subcateg_col] = (avg_price_category - X_df[categ_col]).astype(np.float32)

    return X_df


In [13]:
import numpy as np

# from numeric_utils import compute_concurrently
# from numeric_rolling_features import nmonths_features
# from price_features import get_price_features, get_dollar_value_features


def get_y(sales_df):
    """
    Note that we are not using this month's data in features. we are using previous month's data.
    """
    return sales_df['item_cnt_day']


def date_preprocessing(sales_df):
    sales_df['month'] = sales_df['date_block_num'] % 12 + 1
    sales_df['year'] = (sales_df['date_block_num'] // 12 + 2013).astype(np.int16)


def basic_preprocessing(sales_df):
    date_preprocessing(sales_df)
    sales_df.sort_values(['shop_id', 'item_id', 'date_block_num'], inplace=True)
    shop_id_changed = sales_df.shop_id.diff() != 0
    item_id_changed = sales_df.item_id.diff() != 0
    ids_changed = shop_id_changed | item_id_changed
    sales_df['shop_item_group'] = ids_changed.cumsum()


def get_numeric_rolling_feature_df(sales_df, process_count=4):
    basic_preprocessing(sales_df)

    sales_df['log_p'] = np.log(sales_df['item_price'])
    quantiles = [0.25, 0.5, 0.75, 0.9]

    sales_1M_features_args = [(nmonths_features, (sales_df, 'item_cnt_day', 1, quantiles), {})]
    sales_2M_features_args = [(nmonths_features, (sales_df, 'item_cnt_day', 2, quantiles), {})]
    sales_4M_features_args = [(nmonths_features, (sales_df, 'item_cnt_day', 4, quantiles), {})]

    args = sales_1M_features_args
    args += sales_2M_features_args
    args += sales_4M_features_args

    df = compute_concurrently(args, process_count=process_count).astype('float32')

    df['month'] = sales_df['month'].astype('uint8')
    df['year'] = sales_df['year'] - 2013
    df['shop_id'] = sales_df['shop_id'].astype('uint8')
    df['item_id'] = sales_df['item_id'].astype('uint16')
    df['item_category_id'] = sales_df['item_category_id'].astype('uint8')

    # std on first date is NaN
    print('Number of nan elements', df.isna().sum().sum())
    return df


class NumericFeatures:
    def __init__(self, sales_df, items_df):
        self._sales_df = sales_df
        self._items_df = items_df

        basic_preprocessing(sales_df)

    def get(self, sales_df):
        # assert sales_df[sales_df.item_id.isin([83, 173])].empty

        df = get_numeric_rolling_feature_df(sales_df)
        print('Numeric rolling feature added.')

        # price features
        df['date_block_num'] = sales_df['date_block_num'].astype('uint8')
        df = get_price_features(sales_df, df)
        print('Price features added')

        df = get_dollar_value_features(sales_df, df)
        print('Dollar value features added')

        df.drop('date_block_num', axis=1, inplace=True)

        return df


## Top level code

In [14]:
# from model_validator import ModelValidator
# from rolling_mean_encoding import rolling_mean_encoding
import pandas as pd
from datetime import datetime
# from constants import (DATA_FPATH, TEST_LIKE_SALES_FPATH, SALES_FPATH, ITEMS_FPATH, SHOPS_FPATH, TEST_SALES_FPATH,
#                        ITEM_CATEGORIES_FPATH)

# from numeric_utils import get_date_block_num
# from numeric_features import NumericFeatures, get_y, date_preprocessing
# from id_features import IdFeatures
# from lagged_features import add_lagged_features
# from city_data_features import add_city_data_features


class ModelData:
    EPSILON = 1e-4

    def __init__(self, sales_df, items_df, shops_df):
        self._sales_df = sales_df
        self._items_df = items_df
        self._shops_df = shops_df

        # adding items_category_id to dataframe.
        item_to_cat_dict = self._items_df.set_index('item_id')['item_category_id'].to_dict()
        self._sales_df['item_category_id'] = self._sales_df.item_id.map(item_to_cat_dict)

        self._numeric_features = NumericFeatures(self._sales_df, self._items_df)

        # orig_item_id is needed in id_features.
        self._sales_df['orig_item_id'] = self._sales_df['item_id']
        self._id_features = IdFeatures(self._sales_df, self._items_df)

    def get_train_X_y(self):
        print('Fetching X')
        X_df = self.get_X(self._sales_df)
        print('X fetched. Fetching y')
        y_df = get_y(self._sales_df).to_frame('item_cnt_month')
        print('Y fetched')

        # Retain common rows
        X_df = X_df.join(y_df[[]], how='inner')
        y_df = y_df.join(X_df[[]], how='inner')['item_cnt_month']
        # Order
        y_df = y_df.loc[X_df.index]
        return (X_df, y_df)

    def get_X(self, sales_df):
        X_df = self._numeric_features.get(sales_df)
        assert X_df.index.equals(sales_df.index)
        print('Numeric features fetched.')

        X_df['orig_item_id'] = sales_df['orig_item_id']
        X_df['date_block_num'] = sales_df['date_block_num']

        # # add first month related features for item_id
        X_df = self._id_features.get_fm_features(X_df, item_id_and_shop_id=False)
        # add first month related features for item_id and shop_id jointly
        X_df = self._id_features.get_fm_features(X_df, item_id_and_shop_id=True)

        print('Id features added')

        # lagged features added
        X_df = add_lagged_features(X_df, 'item_cnt_day_1M_sum')
        print('Lagged features added')

        # City related features like area, coordinate, city_id
        X_df = add_city_data_features(X_df, self._shops_df)
        print('City data features added')

        return X_df

    def get_test_X(self, sales_test_df, test_datetime: datetime):

        item_id_original = sales_test_df['item_id'].copy()
        # adding items_category_id to dataframe.
        item_to_cat_dict = self._items_df.set_index('item_id')['item_category_id'].to_dict()
        sales_test_df['item_category_id'] = sales_test_df.item_id.map(item_to_cat_dict)

        sales_test_df['date'] = test_datetime.strftime('%d.%m.%Y')
        sales_test_df['date_block_num'] = get_date_block_num(test_datetime)
        sales_test_df['orig_item_id'] = sales_test_df['item_id']

        sales_test_df['item_cnt_day'] = 0
        sales_test_df['item_price'] = 0

        date_preprocessing(sales_test_df)
        assert sales_test_df.loc[item_id_original.index]['orig_item_id'].equals(item_id_original)

        test_dbn = get_date_block_num(test_datetime)
        recent_sales_df = self._sales_df[self._sales_df.date_block_num < test_dbn]

        recent_sales_df = recent_sales_df.drop('shop_item_group', axis=1)

        subtract_index_offset = max(recent_sales_df.index) - (min(sales_test_df.index) - 1)
        recent_sales_df.index -= subtract_index_offset

        df = pd.concat([recent_sales_df, sales_test_df], axis=0, sort=False)

        assert df.loc[item_id_original.index]['orig_item_id'].equals(item_id_original)

        print('Preprocessing X about to be done now.')
        X_df = self.get_X(df)

        X_df = X_df.loc[sales_test_df.index]

        assert X_df.loc[item_id_original.index]['orig_item_id'].equals(item_id_original)
        return X_df


def mean_encoding_preprocessing(sales):
    test_sales_df = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv', index_col=0)
    test_sales_df['date_block_num'] = 34
    test_sales_df['item_cnt_day'] = 0.33
    index_offset = sales.index.max() + 1
    print('Offsetting test sales index by ', index_offset)
    test_sales_df.index += index_offset
    combined_sales_df = pd.concat(
        [sales[['date_block_num', 'item_id', 'shop_id', 'item_cnt_day']], test_sales_df], sort=True)
    combined_sales_df = pd.merge(
        combined_sales_df.reset_index(), items[['item_id', 'item_category_id']], on='item_id',
        how='left').set_index('index')

    combined_sales_df = rolling_mean_encoding(combined_sales_df)
    train_mean_encoding = combined_sales_df.loc[sales.index].copy()
    test_mean_encoding = combined_sales_df.loc[test_sales_df.index].copy()
    test_mean_encoding.index = test_mean_encoding.index - (sales.index.max() + 1)
    assert test_mean_encoding.shape[0] + train_mean_encoding.shape[0] == combined_sales_df.shape[0]
    assert (test_mean_encoding.date_block_num == 34).all()

    return (train_mean_encoding, test_mean_encoding)


def get_val_dfs(skip_last_n_months: int):
    """
    skip_last_n_months decides how many months prior to Nov 2015 to take as the validation data.
    """
    year = (1 + 33 - skip_last_n_months) // 12 + 2013
    month = (1 + 33 - skip_last_n_months) % 12

    mv = ModelValidator(sales, X_df, y_df, skip_last_n_months=skip_last_n_months)
    val_X_ids, val_y_df = mv.get_val_data()
    print('Before get_test_X', val_X_ids.shape, val_y_df.shape)
    val_X_df = md.get_test_X(val_X_ids, datetime(year, month, 1))
    assert val_X_df.index.equals(val_X_ids.index)
    print('After get_test_X', val_X_df.shape)
    return (val_X_df, val_y_df)




In [15]:
# t_sales = sales[sales.date_block_num >=30].copy()
# md = ModelData(t_sales, items, shops)
md = ModelData(sales, items, shops)
X_df, y_df= md.get_train_X_y()

Fetching X
Number of nan elements 0
Numeric rolling feature added.
Price features added
Dollar value features added
Numeric features fetched.
Id features added
3 Month lagged feature computed.
6 Month lagged feature computed.
12 Month lagged feature computed.
Lagged features added
City data features added
X fetched. Fetching y
Y fetched


In [16]:
assert not X_df.isna().any().any()

In [17]:
# t_test = test[test.shop_id.isin([1,2,3,4,5,6,7,8,9])].copy()
# test_df = md.get_test_X(t_test, datetime(2015,11,1))

test_df = md.get_test_X(test, datetime(2015,11,1))

Preprocessing X about to be done now.


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:32: RuntimeWarning: divide by zero encountered in log


Number of nan elements 0
Numeric rolling feature added.
Price features added
Dollar value features added
Numeric features fetched.
Id features added
3 Month lagged feature computed.
6 Month lagged feature computed.
12 Month lagged feature computed.
Lagged features added
City data features added


In [18]:
test_df.shape

(214200, 54)

In [19]:
assert test_df.loc[test.index].orig_item_id.equals(test.item_id)

In [20]:
print(X_df.shape, y_df.shape)

(8049107, 54) (8049107,)


In [21]:
print(test_df.shape)
print(test.shape)

(214200, 54)
(214200, 11)


# Mean Encoding

In [22]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook


def _rolling_mean_encoding_by_id(combined_sales_df, col_id):
    if col_id == 'item_shop_id':
        assert combined_sales_df.groupby(
            [col_id, 'date_block_num'])['item_cnt_day'].count().shape[0] == combined_sales_df.shape[0]
        df = combined_sales_df[['item_cnt_day', 'item_shop_id', 'date_block_num']].rename(
            {
                'item_cnt_day': 'item_cnt_month'
            }, axis=1)
    else:
        df = combined_sales_df.groupby(
            [col_id, 'date_block_num'])['item_cnt_day'].mean().to_frame('item_cnt_month').reset_index()

    df.sort_values('date_block_num', inplace=True)

    sum_df = df.groupby(col_id)['item_cnt_month'].cumsum() - df['item_cnt_month']
    count_df = df.groupby(col_id)['item_cnt_month'].cumcount()
    id_encoding = sum_df / count_df
    id_encoding[count_df == 0] = -10
    id_encoding = id_encoding.to_frame(col_id + '_enc').astype(np.float32)

    assert id_encoding.index.equals(df.index)
    id_encoding['date_block_num'] = df['date_block_num']
    id_encoding[col_id] = df[col_id]
    return id_encoding


def _rolling_quantile_encoding_by_id(combined_sales_df, col_id, quantile):
    assert col_id != 'item_shop_id'
    df = combined_sales_df.groupby(
        [col_id, 'date_block_num'])['item_cnt_day'].quantile(quantile).to_frame('item_cnt_month').reset_index()

    df.sort_values('date_block_num', inplace=True)

    # Alternate formulation.
    # df.groupby(3)[1].rolling(10, min_periods=1).quantile(0).reset_index(level=0)

    sum_df = df.groupby(col_id)['item_cnt_month'].cumsum() - df['item_cnt_month']
    count_df = df.groupby(col_id)['item_cnt_month'].cumcount()
    id_encoding = sum_df / count_df
    id_encoding[count_df == 0] = -10
    id_encoding = id_encoding.to_frame(col_id + '_qt_{}_enc'.format(quantile)).astype(np.float32)

    assert id_encoding.index.equals(df.index)
    id_encoding['date_block_num'] = df['date_block_num']
    id_encoding[col_id] = df[col_id]
    return id_encoding


def rolling_mean_encoding(combined_sales_df):

    int64 = combined_sales_df[['item_id', 'shop_id', 'item_category_id']].dtypes == np.int64
    int32 = combined_sales_df[['item_id', 'shop_id', 'item_category_id']].dtypes == np.int32
    assert (int32 | int64).all()

    orig_index = combined_sales_df.index
    combined_sales_df.reset_index(inplace=True)

    # item shop_encoding
    combined_sales_df['item_shop_id'] = combined_sales_df['item_id'] * 100 + combined_sales_df['shop_id']
    # shop category id
    combined_sales_df['shop_category_id'] = combined_sales_df['shop_id'] * 100 + combined_sales_df['item_category_id']

    for col_id in ['item_id', 'item_category_id', 'shop_id', 'item_shop_id', 'shop_category_id']:
        # item_id
        encoding = _rolling_mean_encoding_by_id(combined_sales_df, col_id)
        combined_sales_df = pd.merge(combined_sales_df, encoding, on=[col_id, 'date_block_num'], how='left')
        print('{} encoding completed'.format(col_id))

    pbar = tqdm_notebook([0.1, 0.9, 0.95])
    for qt in pbar:
        for col_id in tqdm_notebook(['item_id', 'item_category_id', 'shop_id', 'shop_category_id']):
            pbar.set_description('Running {} Quantile mean encoding for "{}"'.format(qt, col_id))
            # item_id
            encoding = _rolling_quantile_encoding_by_id(combined_sales_df, col_id, qt)
            combined_sales_df = pd.merge(combined_sales_df, encoding, on=[col_id, 'date_block_num'], how='left')
            print('{} encoding completed'.format(col_id))

    combined_sales_df.drop(['item_shop_id', 'shop_category_id'], axis=1, inplace=True)
    combined_sales_df.set_index('index', inplace=True)

    return combined_sales_df.loc[orig_index]


In [23]:
test_sales_df =pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv', index_col=0)
test_sales_df['date_block_num'] = 34
test_sales_df['item_cnt_day'] = 0.33
index_offset = sales.index.max() + 1
print('Offsetting test sales index by ', index_offset)
test_sales_df.index += index_offset
combined_sales_df = pd.concat([sales[['date_block_num','item_id','shop_id','item_cnt_day']], test_sales_df],sort=True)

Offsetting test sales index by  9375832


In [24]:
combined_sales_df = pd.merge(combined_sales_df.reset_index(), items[['item_id','item_category_id']],
                             on='item_id',how='left').set_index('index')


In [25]:
combined_sales_df = rolling_mean_encoding(combined_sales_df)

item_id encoding completed
item_category_id encoding completed
shop_id encoding completed
item_shop_id encoding completed
shop_category_id encoding completed


item_id encoding completed
item_category_id encoding completed
shop_id encoding completed
shop_category_id encoding completed


item_id encoding completed
item_category_id encoding completed
shop_id encoding completed
shop_category_id encoding completed


item_id encoding completed
item_category_id encoding completed
shop_id encoding completed
shop_category_id encoding completed



In [26]:
train_mean_encoding = combined_sales_df.loc[sales.index].copy()
print('Adjusting offseting of test sales by ', sales.index.max() +1)
test_mean_encoding = combined_sales_df.loc[test_sales_df.index].copy()
test_mean_encoding.index = test_mean_encoding.index - (sales.index.max() +1)
assert test_mean_encoding.shape[0] + train_mean_encoding.shape[0] == combined_sales_df.shape[0]
assert (test_mean_encoding.date_block_num == 34).all()

test_mean_encoding.drop(['date_block_num','item_cnt_day','item_id','shop_id','item_category_id'],axis=1,inplace=True)
train_mean_encoding.drop(['date_block_num','item_cnt_day','item_id','shop_id','item_category_id'],axis=1,inplace=True)
del combined_sales_df

Adjusting offseting of test sales by  9375832


In [27]:
assert not test_mean_encoding.isna().any().any()
assert not train_mean_encoding.isna().any().any()

In [28]:
assert not X_df.isna().any().any()
X_df = pd.concat([X_df, train_mean_encoding], axis=1)
assert not X_df.isna().any().any()

In [29]:
assert not test_df.isna().any().any()
test_df = pd.concat([test_df, test_mean_encoding], axis=1)
assert not test_df.isna().any().any()

In [30]:
X_df.to_hdf(DATA_FNAME,'X')
y_df.to_hdf(DATA_FNAME, 'y')

In [31]:
test_df.to_hdf(DATA_FNAME,'test_X')